In [1]:
import sys
from opentree import OT

In [3]:
!wget https://raw.githubusercontent.com/McTavishLab/biodiversity_next/master/example.csv

--2019-12-29 11:42:17--  https://raw.githubusercontent.com/McTavishLab/biodiversity_next/master/example.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.128.133, 151.101.0.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.128.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49361 (48K) [text/plain]
Saving to: ‘example.csv.4’

example.csv.4       100%[===================>]  48.20K  86.9KB/s    in 0.6s    

2019-12-29 11:42:18 (86.9 KB/s) - ‘example.csv.4’ saved [49361/49361]



In [4]:
!head example.csv

gbifID	datasetKey	occurrenceID	kingdom	phylum	class	order	family	genus	species	infraspecificEpithet	taxonRank	scientificName	verbatimScientificName	verbatimScientificNameAuthorship	countryCode	locality	stateProvince	occurrenceStatus	individualCount	publishingOrgKey	decimalLatitude	decimalLongitude	coordinateUncertaintyInMeters	coordinatePrecision	elevation	elevationAccuracy	depth	depthAccuracy	eventDate	day	month	year	taxonKey	speciesKey	basisOfRecord	institutionCode	collectionCode	catalogNumber	recordNumber	identifiedBy	dateIdentified	license	rightsHolder	recordedBy	typeStatus	establishmentMeans	lastInterpreted	mediaType	issue
2423004790	50c9509d-22c7-4a22-a47d-8c48425ef4a7	https://www.inaturalist.org/observations/32478397	Animalia	Arthropoda	Insecta	Odonata	Libellulidae	Tramea	Tramea lacerata		SPECIES	Tramea lacerata Hagen, 1861	Tramea lacerata		US		California			28eb1a3f-1c15-4a95-931a-4af90ecb574d	37.36422	-120.424003	12.0						2019-09-10T08:39:35Z	10	9	2019	1428475	1428475	HUMAN_OB

In [5]:
filename = "example.csv"
fi = open(filename)
header = fi.readline().split('\t')

In [6]:
#Get indexes for each column in the csv file
col_dict = {}
for i, col in enumerate(header):
    col_dict[col] = i

In [7]:
col_dict

{'gbifID': 0,
 'datasetKey': 1,
 'occurrenceID': 2,
 'kingdom': 3,
 'phylum': 4,
 'class': 5,
 'order': 6,
 'family': 7,
 'genus': 8,
 'species': 9,
 'infraspecificEpithet': 10,
 'taxonRank': 11,
 'scientificName': 12,
 'verbatimScientificName': 13,
 'verbatimScientificNameAuthorship': 14,
 'countryCode': 15,
 'locality': 16,
 'stateProvince': 17,
 'occurrenceStatus': 18,
 'individualCount': 19,
 'publishingOrgKey': 20,
 'decimalLatitude': 21,
 'decimalLongitude': 22,
 'coordinateUncertaintyInMeters': 23,
 'coordinatePrecision': 24,
 'elevation': 25,
 'elevationAccuracy': 26,
 'depth': 27,
 'depthAccuracy': 28,
 'eventDate': 29,
 'day': 30,
 'month': 31,
 'year': 32,
 'taxonKey': 33,
 'speciesKey': 34,
 'basisOfRecord': 35,
 'institutionCode': 36,
 'collectionCode': 37,
 'catalogNumber': 38,
 'recordNumber': 39,
 'identifiedBy': 40,
 'dateIdentified': 41,
 'license': 42,
 'rightsHolder': 43,
 'recordedBy': 44,
 'typeStatus': 45,
 'establishmentMeans': 46,
 'lastInterpreted': 47,
 'medi

In [8]:
match_dict = {}
ott_ids = set()

#Loop through each line in the gbif output
for lin in fi:
    lii = lin.split('\t')
    gb_id = lii[col_dict['taxonKey']]
    sys.stdout.write(".") #progress bar
    sys.stdout.flush()
    if gb_id in match_dict:
        #Skip gb_id's you have already matched
        pass
    else:
        # Do a driect match to gbif id's in the open tree taxnomy
        gbiftax = "gbif:{}".format(int(gb_id))
        res = OT.taxon_info(source_id=gbiftax)
        if res.status_code == 200:
            ott_id = int(res.response_dict['ott_id'])
            match_dict[gb_id] = ott_id
        if res.status_code == 400:
            # If GBIF id isn' found in the open tree taxonomy, search on scientific name
            spp_name = lii[col_dict['verbatimScientificName']]
            sys.stdout.write("{},{} not matched on ID\n".format(gbiftax, spp_name))
            res2 = OT.tnrs_match([spp_name])
            if res2.status_code == 200:
                if len(res2.response_dict['results']) > 0:
                    ott_id = int(res2.response_dict['results'][0]['matches'][0]['taxon']['ott_id'])
                    match_dict[gb_id] = ott_id
                    sys.stdout.write("{},{} matched on name to ott id{}\n".format(gbiftax, spp_name, ott_id))
                else:
                    sys.stdout.write("{},{} still NO MATCH\n".format(gbiftax, spp_name))
                    match_dict[gb_id] = None
        ott_ids.add(ott_id)

........................gbif:5229155,Pelecanus erythrorhynchos not matched on ID
gbif:5229155,Pelecanus erythrorhynchos matched on name to ott id316989
....gbif:9088491,Dryobates nuttallii not matched on ID
gbif:9088491,Dryobates nuttallii matched on name to ott id701703
..gbif:5231677,Mimus polyglottos not matched on ID
gbif:5231677,Mimus polyglottos matched on name to ott id571310
......................................gbif:2498167,Anser caerulescens not matched on ID
gbif:2498167,Anser caerulescens matched on name to ott id190878
...........gbif:2498161,Anser rossii not matched on ID
gbif:2498161,Anser rossii matched on name to ott id767830
....................

In [9]:
len(ott_ids)

56

In [19]:
trefile = "names.tre"
#Get the syntehtic tree from OpenTreem and withe the ciattions to a text file.
output = OT.synth_induced_tree(ott_ids=list(ott_ids),  label_format='id')
output.tree.write(path = "names.tre", schema = "newick")
sys.stdout.write("Tree written to {}\n".format(trefile))

Tree written to names.tre


NewickError: Unexpected newick format 'ott3599326' 
You may want to check other newick loading flags like 'format' or 'quoted_node_names'.

In [11]:
studies = output.response_dict['supporting_studies']

In [21]:
for study in studies:
    studyid = study.split('@')[0]
    studyres = OT.find_studies(studyid, search_property = 'ot:studyId', verbose="True")
    new_cite = studyres.response_dict.get('matched_studies', None)
    if new_cite:
        print(new_cite[0].get('ot:studyPublicationReference', '') + '\n' + new_cite[0].get('ot:studyPublication', '') + '\n')



Carrano M.T., Benson R.B., & Sampson S.D. 2012. The phylogeny of Tetanurae (Dinosauria: Theropoda). Journal of Systematic Palaeontology, 10(2): 211-300.
http://dx.doi.org/10.1080/14772019.2011.630927

Brusatte S.L., Benton M.J., Desojo J.B., & Langer M.C. 2010. The higher-level phylogeny of Archosauria (Tetrapoda: Diapsida). Journal of Systematic Palaeontology, 8(1): 3-47.
http://dx.doi.org/10.1080/14772010903537732

Andres B.B., Clark J.M., & Xu X. 2012. A new rhamphorhynchid pterosaur from the Upper Jurassic of Xinjiang, China, and the phylogenetic relationships of basal pterosaurs. Journal of Vertebrate Paleontology, 30(1): 163-187.
http://dx.doi.org/10.1080/02724630903409220

Zanno L.E. 2010. A taxonomic and phylogenetic re-evaluation of Therizinosauria (Dinosauria: Maniraptora). Journal of Systematic Palaeontology, 8(4): 37-41.
http://dx.doi.org/10.1080/14772019.2010.488045

Dongyu H., Lianhai H., Lijun Z., & Xing X. 2010. A pre-Archaeopteryx troodontid theropod from China with lo

Pyron, R.A., & Wiens J.J. 2011. A large-scale phylogeny of Amphibia including over 2800 species, and a revised classification of extant frogs, salamanders, and caecilians. Molecular Phylogenetics Evolution 61 (2): 543-583.
http://dx.doi.org/10.1016/j.ympev.2011.06.012

Selvatti, Alexandre Pedro, Luiz Pedreira Gonzaga, Claudia Augusta de Moraes Russo. 2015. A Paleogene origin for crown passerines and the diversification of the Oscines in the New World. Molecular Phylogenetics and Evolution 88: 1-15.
http://dx.doi.org/10.1016/j.ympev.2015.03.018

Moyle, Robert G., Carl H. Oliveros, Michael J. Andersen, Peter A. Hosner, Brett W. Benz, Joseph D. Manthey, Scott L. Travers, Rafe M. Brown, Brant C. Faircloth. 2016. Tectonic collision and uplift of Wallacea triggered the global songbird radiation. Nature Communications 7: 12709
http://dx.doi.org/10.1038/ncomms12709

Odeen, A., Hastad O., & Alstrom P. 2011. Evolution of ultraviolet vision in the largest avian radiation - the passerines. BMC Evo

Wang, Ning, Rebecca T. Kimball, Edward L. Braun, Bin Liang, Zhengwang Zhang. 2013. Assessing phylogenetic relationships among Galliformes: a multigene phylogeny with expanded taxon sampling in Phasianidae. PLoS ONE 8 (5): e64312.
http://dx.doi.org/10.1371/journal.pone.0064312

Hosner, Peter A. Brant C. Faircloth, Travis C. Glenn, Edward L. Braun, Rebecca T. Kimball. 2015. Avoiding missing data biases in phylogenomic inference: an empirical study in the landfowl (Aves: Galliformes). Molecular Biology and Evolution 33 (4): 1110-1125
http://dx.doi.org/10.1093/molbev/msv347

Worthy T., Mitri M., Handley W.D., Lee M.S., Anderson A., & Sand C. 2016. Osteology Supports a Stem-Galliform Affinity for the Giant Extinct Flightless Bird Sylviornis neocaledoniae (Sylviornithidae, Galloanseres). PLoS ONE, 11(3): e0150871.
http://dx.doi.org/10.1371/journal.pone.0150871

Lerner, H., M. Klaver, D. Mindell. 2008. Molecular phylogenetics of the Buteonine birds of prey (Accipitridae). The Auk 125 (2): 304

Besnard, Guillaume, Joris A. M. Bertrand, Boris Delahaie, Yann X. C. Bourgeois, Emeline Lhuillier, Christophe Thébaud. 2015. Valuing museum specimens: high-throughput DNA sequencing on historical collections of New Guinea crowned pigeons (Goura). Biological Journal of the Linnean Society 117 (1): 71-82
http://dx.doi.org/10.1111/bij.12494

Pereira, S., Johnson K., Clayton D., and Baker A. 2007. Mitochondrial and nuclear DNA sequences support a Cretaceous origin of Columbiformes and a dispersal-driven radiation in the Paleogene. Systematic Biology 56 (4): 656-672.
http://dx.doi.org/10.1080/10635150701549672

Fuchs, Jérôme, Steven Chen, Jeff A. Johnson, David P. Mindell. 2011. Pliocene diversification within the South American forest falcons (Falconidae: Micrastur). Molecular Phylogenetics and Evolution 60 (3): 398-407.
http://dx.doi.org/10.1016/j.ympev.2011.05.008

Fuchs, Jérôme, Jeff A. Johnson, David P. Mindell. 2012. Molecular systematics of the caracaras and allies (Falconidae: Polyb

In [15]:
studyid

'ot_1510'

In [20]:
new_cite

[{'ot:studyId': 'pg_2853'}]